## Graph structure

In [1]:
class edgestore:

    def __init__(self):
        self._store     = {} # neighbours dictionary
        self._num_edges = 0
        self._edge_set  = set()

    # adding edge u - v updating neighbours dictionary
    def add(self,u,v):
        if u in self._store.keys():
            self._store[u].add(v)
        else :
            self._store[u] = set([v])

        if v in self._store.keys():
            self._store[v].add(u)
        else :
            self._store[v] = set([u])
        
        self._num_edges += 1
        self._edge_set.add((u,v))


    # deleting edge u - v updating neighbours dictionary
    def delete(self,u,v):
        self._store[u].remove(v)
        set_u = self._store[u]
        if len(set_u) == 0:
            del self._store[u]

        self._store[v].remove(u)
        set_v = self._store[v]
        if len(set_v) == 0:
            del self._store[v]
        
        if self._num_edges > 0:
            self._num_edges -= 1
            
        self._edge_set.remove((u, v))


    def get_neighbours(self,u):
        return self._store[u]

    def get_vertice_list(self):
        return self._store.keys()

    def get_edges(self):
        return list(self._edge_set)


## Reservoir sampling algorithm for edges

In [2]:
import random

def reservoir_edge(edge,i):
    
    # keep first M items
    if i <= M:
        return True
    
    # with probability M/i keep item i and discard an old one randomly 
    else:
        coin_toss = random.random()
        if coin_toss < (M/i):
            edge_list = S.get_edges()
            num_edges = len(edge_list)
            del_idx = random.randint(0, num_edges-1)
            u1, v1 = edge_list[del_idx]
            S.delete(u1, v1)
            return True

    return False


## Triest algorithm

In [4]:

def update_counters(i, u, v):
        global global_T
        vertices = S.get_vertice_list()
        if u not in vertices or v not in vertices:
            return
        neighbourhood_u = S.get_neighbours(u)
        neighbourhood_v = S.get_neighbours(v)

        shared_neigbourhood = neighbourhood_u & neighbourhood_v
        shared_value = len(shared_neigbourhood)

        if shared_value == 0:
            return
        
        weight_i = max(1, ((i-1)*(i-2))/(M * (M-1)))
        
        for c in shared_neigbourhood:
                local_T[c] += weight_i
                local_T[u] += weight_i
                local_T[v] += weight_i
                global_T   += weight_i              
    
def triest(datafile):
    i = 0
    for line in datafile:
        edge = line.split()
        u = int(edge[0])
        v = int(edge[1])
        if u == v:
            continue
        if u > v:
            tmp = u
            u = v
            v = tmp
        if (u,v) in S.get_edges(): # skip if edge already in sample
            continue
        
        i = i + 1
        update_counters(i,u,v)
        if reservoir_edge((u,v),i):
            S.add(u,v)

    return int(global_T) # floor the number


## Test algorithms

In [12]:
from collections import defaultdict

M = 200 #more than 2500 is stable to 326
S = edgestore()

local_T = defaultdict(lambda:0)
global_T = 0

# dataset found from the ones in canvas 
#1226 vertices #2615 edges #326 triangles

file = open('data/out.maayan-faa', 'r') 
result = triest(file)

print("M = ", M)
print("Global Triangles = ", result)


M =  200
Global Triangles =  274


In [10]:
#local_T